# 学习PyTorch-Day3

In [2]:
import torch
import numpy as np

# for mac
device = torch.device("mps")

# for cuda
# device = torch.device("cuda")

## Transforms-变换

数据并不总是按照机器学习算法所需的形式出现，我们经常需要对数据执行变换，使其适合训练

In [5]:
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(root="data", 
                           train=True,
                           download=False,
                           transform=ToTensor())

### ToTensor()

ToTensor可以将一张图像或者Numpy数组转换成 __FloatTensor__ ，并将图像的像素强度值缩放在[0., 1.] 范围内

### Lambda变换

Lambda可以封装任何自定义的变换函数

## 构建神经网络

神经网络由各种层/模块构成，这些层/模块包装了对数据的操作
- __torch.nn__ 中包括了构建神经网络所需的各种模板，PyTorch模板中的每个模块都是 __nn.Module__ 的子集

### 实例：构建分类网络，对FashionMNIST数据集进行分类

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 指认设备
device = torch.device(
    "cuda" 
    if torch.cuda.is_available()
    else "mps" 
    if torch.backends.mps.is_available()
    else "cpu"
)

print(device)

mps


- 定义神经网络（继承于 __nn.Module__）

In [7]:
from turtle import forward


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # 展平处理
        self.flatten = nn.Flatten()
        # 全连接+激活
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

- 实例化网络，指认到设备上

In [8]:
model = NeuralNetwork()
model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

- 初始化输入
- 因为此问题是互斥分类问题，即结果只可能属于一类，所以应使用 Softmax 激活函数
- nn.Softmax(dim=1)，此处的 dim 表示在哪个维度上 和为1，此处应该是10类输出之和1，所以应为 dim=1
- Tensor.argmax() == torch.argmax(Tensor, dim)，返回输入Tensor中最大值的下标

In [32]:
t = torch.rand(1, 28, 28, device=device)
out = model(t)
# 因为此问题是互斥分类问题，即结果只可能属于一类，所以应使用 Softmax 激活函数
softmax_func = nn.Softmax(dim=1)  # 此处的 dim 表示在哪个维度上 和为1，此处应该是10类输出之和1，所以应为 dim=1
pred_prob = softmax_func(out)
# argmax
pred_cls = torch.argmax(pred_prob, dim=1)
print(pred_cls.item())

7
